##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初学者的 TensorFlow 2.0 教程

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/quickstart/beginner"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 观看</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

此简短介绍使用 [Keras](https://tensorflow.google.cn/guide/keras/overview) 进行以下操作：

1. 加载一个预构建的数据集。
2. 构建对图像进行分类的神经网络机器学习模型。
3. 训练此神经网络。
4. 评估模型的准确率。

这是一个 [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) 笔记本文件。 Python程序可以直接在浏览器中运行，这是学习 Tensorflow 的绝佳方式。想要学习该教程，请点击此页面顶部的按钮，在Google Colab中运行笔记本。

1. 在 Colab中, 连接到Python运行环境： 在菜单条的右上方, 选择 *CONNECT*。
2. 运行所有的代码块: 选择 *Runtime* &gt; *Run all*。

## 设置 TensorFlow

首先将 TensorFlow 导入到您的程序：

In [2]:
import tensorflow as tf

如果您在自己的开发环境而不是 [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb) 中操作，请参阅设置 TensorFlow 以进行开发的[安装指南](https://tensorflow.google.cn/install)。

注：如果您使用自己的开发环境，请确保您已升级到最新的 `pip` 以安装 TensorFlow 2 软件包。有关详情，请参阅[安装指南](https://tensorflow.google.cn/install)。

## 加载数据集

加载并准备 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。将样本数据从整数转换为浮点数：

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 构建机器学习模型

通过堆叠层来构建 `tf.keras.Sequential` 模型。

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

对于每个样本，模型都会返回一个包含 [logits](https://developers.google.com/machine-learning/glossary#logits) 或 [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) 分数的向量，每个类一个。

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.13613781, -0.53124315, -0.2981916 , -0.24647225, -0.47956654,
         0.5498646 ,  0.09666567,  0.22561511, -0.04717921, -0.18241784]],
      dtype=float32)

`tf.nn.softmax` 函数将这些 logits 转换为每个类的*概率*： 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.11750696, 0.06028701, 0.07610913, 0.08014901, 0.06348434,
        0.17772268, 0.11295906, 0.1285059 , 0.09782504, 0.0854509 ]],
      dtype=float32)

注：可以将 `tf.nn.softmax` 烘焙到网络最后一层的激活函数中。虽然这可以使模型输出更易解释，但不建议使用这种方式，因为在使用 softmax 输出时不可能为所有模型提供精确且数值稳定的损失计算。 

使用 `losses.SparseCategoricalCrossentropy` 为训练定义损失函数，它会接受 logits 向量和 `True` 索引，并为每个样本返回一个标量损失。

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

此损失等于 true 类的负对数概率：如果模型确定类正确，则损失为零。

这个未经训练的模型给出的概率接近随机（每个类为 1/10），因此初始损失应该接近 `-tf.math.log(1/10) ~= 2.3`。

In [8]:
loss_fn(y_train[:1], predictions).numpy()

1.727531

在开始训练之前，使用 Keras `Model.compile` 配置和编译模型。将 [`optimizer`](https://tensorflow.google.cn/api_docs/python/tf/keras/optimizers) 类设置为 `adam`，将 `loss` 设置为您之前定义的 `loss_fn` 函数，并通过将 `metrics` 参数设置为 `accuracy` 来指定要为模型评估的指标。

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 训练并评估模型

使用 `Model.fit` 方法调整您的模型参数并最小化损失： 

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
 133/1875 [=>............................] - ETA: 0s - loss: 0.8907 - accuracy: 0.7448      

2024-07-10 12:30:42.829614: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 1s 378us/step - loss: 0.3003 - accuracy: 0.9117
Epoch 2/5
1875/1875 [==============================] - 1s 372us/step - loss: 0.1441 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 1s 374us/step - loss: 0.1080 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 1s 373us/step - loss: 0.0874 - accuracy: 0.9727
Epoch 5/5
1875/1875 [==============================] - 1s 375us/step - loss: 0.0762 - accuracy: 0.9765


`Model.evaluate` 方法通常在 "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" 或 "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)" 上检查模型性能。

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0737 - accuracy: 0.9771 - 108ms/epoch - 345us/step


[0.07374721765518188, 0.9771000146865845]

现在，这个照片分类器的准确度已经达到 98%。想要了解更多，请阅读 [TensorFlow 教程](https://tensorflow.google.cn/tutorials/)。

如果您想让模型返回概率，可以封装经过训练的模型，并将 softmax 附加到该模型：

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.1274221e-07, 1.3944246e-09, 3.6083061e-06, 3.1712349e-05,
        1.0723298e-10, 6.1581551e-07, 1.0641497e-13, 9.9996114e-01,
        1.3410575e-07, 2.6433925e-06],
       [2.7423801e-09, 8.1576582e-05, 9.9990630e-01, 9.5986916e-06,
        5.1563774e-14, 1.7583911e-06, 7.9305778e-08, 1.1695698e-13,
        6.4428832e-07, 8.6933759e-11],
       [5.6400938e-07, 9.9744368e-01, 3.4890484e-04, 3.4063673e-05,
        9.0967973e-05, 1.5468780e-05, 1.2700001e-05, 1.8597718e-03,
        1.8843610e-04, 5.5217629e-06],
       [9.9971682e-01, 6.2276602e-09, 9.1739508e-05, 9.3510636e-07,
        1.6178970e-05, 1.3211679e-05, 1.5273837e-04, 4.0274213e-06,
        2.8612030e-07, 3.9803035e-06],
       [1.1897397e-06, 5.9155575e-10, 4.4175889e-07, 1.3088122e-08,
        9.9846482e-01, 2.5364558e-08, 7.0111312e-08, 4.6791298e-05,
        3.7394784e-07, 1.4862340e-03]], dtype=float32)>

## 结论

恭喜！您已经利用 [Keras](https://tensorflow.google.cn/guide/keras/overview) API 借助预构建数据集训练了一个机器学习模型。

有关使用 Keras 的更多示例，请查阅[教程](https://tensorflow.google.cn/tutorials/keras/)。要详细了解如何使用 Keras 构建模型，请阅读[指南](https://tensorflow.google.cn/guide/keras)。如果您想详细了解如何加载和准备数据，请参阅有关[图像数据加载](https://tensorflow.google.cn/tutorials/load_data/images)或 [CSV 数据加载](https://tensorflow.google.cn/tutorials/load_data/csv)的教程。
